# This is a mostly non-technical demo notebook for a CNN model that predicts March Madness games
##### Below are imports, more than are in use, just extra tools in case.

In [1]:
from __future__ import print_function
import sys
import csv
import tensorflow as tf
import sklearn
import en_core_web_sm
import keras
from keras import layers
from keras import backend as K
from keras.preprocessing.sequence import pad_sequences 
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras_tqdm import TQDMNotebookCallback
from keras.models import Sequential
from keras.layers import Dropout, Activation
from keras.optimizers import SGD
from IPython import display
from keras.preprocessing.sequence import pad_sequences
import warnings
from collections import Counter
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
%matplotlib inline
from matplotlib import rcParams
rcParams['font.family'] = 'monospace'
from matplotlib.ticker import MaxNLocator
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, roc_auc_score, roc_curve, log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
import os, eli5, shap
np.set_printoptions(threshold=sys.maxsize)
from eli5.sklearn import PermutationImportance
from pdpbox import pdp

Using TensorFlow backend.


In [17]:
#Create dataframes that will be used for performance comparison later
pastModels=[[2017,1,.43857],[2017,2,.44981],[2017,3,.45373],[2017,4,.46100],[2017,5,.46107],[2018,1,.53194],[2018,2,.53693],[2018,3,.54013],[2018,4,.54967],[2018,5,.54987],[2019,1,.41477],[2019,2,.42012],[2019,3,.42698],[2019,4,.42788],[2019,5,.43148]]
cols=['Year','Rank','LogLoss']
pastScores=pd.DataFrame(pastModels, columns=cols)
pastMadness=[[2017,65.7,82,93,83],[2018,57,81,131,99.2],[2019,74,124,162,99.99]]
cols=['Year','avg_score','higherSeed_score','myScore','percentile_myScore']
pastMadness=pd.DataFrame(pastMadness, columns=cols)

## Importing the data here and slicing it in different ways for training and testing model
##### We will only be showing testing model performance

In [20]:
tourney_training_data=pd.read_csv("tourney_game_master1.csv")
#Get Training data
tourney_training_data.dropna(inplace=True)
tourneyInputData=tourney_training_data[['A_KP_AdjEM', 'A_KP_SOS_Adj', 'ALuck', 'Asos', 'Ats_pct', 'Aie', 'Afour', 'Amov', 'APOM', 'ASAG', 'AMOR', 'Aelo', 'Awins_top25', 'Awins_top5', 'Awin_pct', 'AAdjustD', 'AAdjustO', 'B_KP_AdjEM', 'B_KP_SOS_Adj', 'BLuck', 'Bsos', 'Bts_pct', 'Bie', 'Bfour', 'Bmov', 'BPOM', 'BSAG', 'BMOR', 'Belo', 'Bwins_top25', 'Bwins_top5', 'Bwin_pct', 'BAdjustD', 'BAdjustO','KP_AdjEM_diff', 'KP_SOS_Adj_diff', 'Luck_diff', 'sos_diff', 'ts_pct_diff', 'ie_diff', 'four_diff', 'mov_diff', 'POM_diff', 'SAG_diff', 'MOR_diff',  'elo_diff', 'wins_top25_diff', 'wins_top5_diff', 'win_pct_diff','AdjustD_diff','AdjustO_diff','Season','ATeam','BTeam']]

#held while scaling done

tourneyOutputData=tourney_training_data[['Season','AWon']]
train_input=tourneyInputData[tourneyInputData['Season']<2017].reset_index(drop=True)
train_input=train_input[['A_KP_AdjEM', 'A_KP_SOS_Adj', 'ALuck', 'Asos', 'Ats_pct', 'Aie', 'Afour', 'Amov', 'APOM', 'ASAG', 'AMOR', 'Aelo', 'Awins_top25', 'Awins_top5', 'Awin_pct', 'AAdjustD', 'AAdjustO', 'B_KP_AdjEM', 'B_KP_SOS_Adj', 'BLuck', 'Bsos', 'Bts_pct', 'Bie', 'Bfour', 'Bmov', 'BPOM', 'BSAG', 'BMOR', 'Belo', 'Bwins_top25', 'Bwins_top5', 'Bwin_pct', 'BAdjustD', 'BAdjustO','KP_AdjEM_diff', 'KP_SOS_Adj_diff', 'Luck_diff', 'sos_diff', 'ts_pct_diff', 'ie_diff', 'four_diff', 'mov_diff', 'POM_diff', 'SAG_diff', 'MOR_diff',  'elo_diff', 'wins_top25_diff', 'wins_top5_diff', 'win_pct_diff','AdjustD_diff','AdjustO_diff']]
train_output=tourneyOutputData[tourneyOutputData['Season']<2017].reset_index(drop=True)
train_output=train_output[['AWon']]


#Get test data
testTemp =tourneyInputData[tourneyInputData['Season']>2016].reset_index(drop=True)
testTemp=testTemp[['Season','ATeam','BTeam']]  
test_input=tourneyInputData[tourneyInputData['Season']>2016].reset_index(drop=True)
test_output=tourneyOutputData[tourneyOutputData['Season']>2016].reset_index(drop=True)


## Here we have a sample of some rows that will be used as inputs to predict the outcome of some March Madness games
##### Does not show all inputs, these inputs are an array of end of season stats for each team and then a comparison differential of said stats
###### Below we also have they testing output data which has the actual outcomes we will be trying to predict

In [21]:
#Show sample of DataFrame
test_input.head(5)

,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,Aie,Afour,Amov,APOM,ASAG,AMOR,Aelo,Awins_top25,Awins_top5,Awin_pct,AAdjustD,AAdjustO,B_KP_AdjEM,B_KP_SOS_Adj,BLuck,Bsos,Bts_pct,Bie,Bfour,Bmov,BPOM,BSAG,BMOR,Belo,Bwins_top25,Bwins_top5,Bwin_pct,BAdjustD,BAdjustO,KP_AdjEM_diff,KP_SOS_Adj_diff,Luck_diff,sos_diff,ts_pct_diff,ie_diff,four_diff,mov_diff,POM_diff,SAG_diff,MOR_diff,elo_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjustD_diff,AdjustO_diff,Season,ATeam,BTeam
0,23.31,7.13,0.046,16.198486,57.770923,62.607934,0.367951,10.476471,20,18,21,1564.338877,3,0,0.882353,95.1,118.4,0.04,-5.66,0.048,5.296888,57.232652,54.037998,0.358434,3.984962,173,176,156,1203.480859,0,0,0.678571,103.5,103.6,23.27,12.79,-0.002,10.901598,0.538271,8.569936,0.009518,6.491508,-153,-158,-135,360.858019,3,0,0.203782,-8.4,14.8,2017,arizona,north-dakota
1,23.31,7.13,0.046,16.198486,57.770923,62.607934,0.367951,10.476471,20,18,21,1564.338877,3,0,0.882353,95.1,118.4,23.67,0.41,0.030,25.407017,60.345084,67.213811,0.371236,15.026786,14,25,26,1347.875927,0,0,0.875000,95.0,118.7,-0.36,6.72,0.016,-9.208531,-2.574161,-4.605877,-0.003284,-4.550315,6,-7,-5,216.462950,3,0,0.007353,0.1,-0.3,2017,arizona,saint-marys
2,16.25,8.37,0.086,8.179100,55.557028,56.305476,0.357436,6.174118,38,37,40,1011.700036,1,0,0.735294,99.8,116.1,13.63,9.65,0.058,4.343330,54.148320,53.293282,0.351570,3.139881,53,49,34,1041.147358,2,0,0.656250,95.8,109.4,2.62,-1.28,0.028,3.835770,1.408708,3.012194,0.005866,3.034237,-15,-12,6,-29.447322,-1,0,0.079044,4.0,6.7,2017,arkansas,seton-hall
3,7.89,-4.52,0.063,13.497683,57.716089,61.772175,0.386777,9.066667,88,97,177,1640.635822,0,0,0.833333,102.6,110.5,24.23,13.08,0.023,10.691229,54.820036,59.018848,0.366041,7.106183,13,14,20,1024.049588,6,0,0.774194,92.9,117.1,-16.34,-17.60,0.040,2.806454,2.896053,2.753328,0.020736,1.960484,75,83,157,616.586233,-6,0,0.059140,9.7,-6.6,2017,new-mexico-st,baylor
4,13.45,5.49,0.079,8.305611,55.716894,55.959719,0.348941,5.568182,61,55,52,982.925540,2,0,0.727273,100.7,114.2,24.23,13.08,0.023,10.691229,54.820036,59.018848,0.366041,7.106183,13,14,20,1024.049588,6,0,0.774194,92.9,117.1,-10.78,-7.59,0.056,-2.385619,0.896858,-3.059129,-0.017100,-1.538001,48,41,32,-41.124048,-4,0,-0.046921,7.8,-2.9,2017,usc,baylor


In [22]:
test_output.head(5)



,Season,AWon
0,2017,1
1,2017,1
2,2017,1
3,2017,0
4,2017,0


## This scales all the data for inputs
##### It uses all tournament data collected to calculate standard deviation and then scales the test data as a subset putting mean at 0

In [23]:
#scale data
scaler = StandardScaler()

scaler.fit(train_input) 
X_train = pd.DataFrame(scaler.transform(train_input), index=train_input.index, columns=train_input.columns)
X_test = pd.DataFrame(scaler.transform(test_input.iloc[:,0:51]), index=test_input.iloc[:,0:51].index, columns=test_input.iloc[:,0:51].columns)
Y_train=train_output
Y_test=test_output

X_test['Season']=testTemp['Season']
X_test['ATeam'] =testTemp['ATeam']
X_test['BTeam']=testTemp['BTeam']


In [24]:
X_test.head(5)

,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,Aie,Afour,Amov,APOM,ASAG,AMOR,Aelo,Awins_top25,Awins_top5,Awin_pct,AAdjustD,AAdjustO,B_KP_AdjEM,B_KP_SOS_Adj,BLuck,Bsos,Bts_pct,Bie,Bfour,Bmov,BPOM,BSAG,BMOR,Belo,Bwins_top25,Bwins_top5,Bwin_pct,BAdjustD,BAdjustO,KP_AdjEM_diff,KP_SOS_Adj_diff,Luck_diff,sos_diff,ts_pct_diff,ie_diff,four_diff,mov_diff,POM_diff,SAG_diff,MOR_diff,elo_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjustD_diff,AdjustO_diff,Season,ATeam,BTeam
0,0.573221,0.228608,0.657582,0.792645,1.132519,0.724414,0.191107,0.654301,-0.417829,-0.447897,-0.414909,0.641486,0.256006,-0.556471,1.303652,0.027134,0.894347,-2.160030,-2.256284,0.708853,-1.111415,0.823014,-1.166051,-0.486481,-0.987405,2.759516,2.803389,2.225919,-0.730274,-1.099018,-0.53908,-0.759788,1.766487,-1.671322,1.856321,1.707803,-0.018722,1.331587,0.204320,1.323864,0.525088,1.144446,-2.252208,-2.326517,-1.868418,0.984805,0.895360,-0.036743,1.449231,-1.188818,1.780350,2017,arizona,north-dakota
1,0.573221,0.228608,0.657582,0.792645,1.132519,0.724414,0.191107,0.654301,-0.417829,-0.447897,-0.414909,0.641486,0.256006,-0.556471,1.303652,0.027134,0.894347,0.620035,-1.089147,0.292258,2.416441,2.192898,1.744238,0.408207,1.813704,-0.526746,-0.287521,-0.304587,-0.172383,-1.099018,-0.53908,1.207704,0.041853,0.960215,-0.015583,0.906863,0.269134,-1.109357,-0.795080,-0.705997,-0.173218,-0.796382,0.057547,-0.129581,-0.085779,0.580750,0.895360,-0.036743,0.071174,-0.009504,-0.037048,2017,arizona,saint-marys
2,-0.224446,0.463203,1.550518,-0.578770,0.132103,-0.653210,-0.562810,-0.420664,-0.068412,-0.077132,-0.054993,-1.534453,-0.639769,-0.556471,-0.162960,0.952762,0.498921,-0.561169,0.687516,0.940294,-1.278695,-0.534502,-1.330545,-0.966160,-1.201786,0.279318,0.203749,-0.148863,-1.357474,-0.232025,-0.53908,-0.983367,0.204172,-0.660533,0.220485,-0.148741,0.461038,0.473945,0.483817,0.467640,0.325921,0.536759,-0.247515,-0.202327,0.065059,-0.107371,-0.257404,-0.036743,0.574128,0.531593,0.805454,2017,arkansas,seton-hall
3,-1.168992,-1.975445,1.037080,0.330774,1.107740,0.541729,1.540877,0.302054,0.902189,1.093706,2.540189,0.941895,-1.087656,-0.556471,0.814782,1.504200,-0.463855,0.685919,1.347035,0.130249,-0.165103,-0.238858,-0.065875,0.045182,-0.195608,-0.547415,-0.512687,-0.421379,-1.423533,1.501959,-0.53908,0.197994,-0.384233,0.681377,-1.281474,-2.302173,0.652943,0.349008,0.961403,0.427759,1.136989,0.348025,1.059894,1.179851,2.135662,1.700400,-1.698359,-0.036743,0.434488,1.322427,-0.795301,2017,new-mexico-st,baylor
4,-0.540801,-0.081662,1.394254,-0.557135,0.204344,-0.728788,-1.171907,-0.572060,0.378064,0.274120,0.172322,-1.647748,-0.191881,-0.556471,-0.242957,1.130010,0.172265,0.685919,1.347035,0.130249,-0.165103,-0.238858,-0.065875,0.045182,-0.195608,-0.547415,-0.512687,-0.421379,-1.423533,1.501959,-0.53908,0.197994,-0.384233,0.681377,-0.841026,-0.981349,0.908815,-0.281200,0.319463,-0.467706,-0.926817,-0.266906,0.667671,0.568782,0.421587,-0.140046,-1.121977,-0.036743,-0.309587,1.058816,-0.349978,2017,usc,baylor


## Loading Model Framework and Weights
##### This loads a file that has the saved framework and tuned model, it then loads the weights which will be used for predicting games

In [25]:
#Load Model and weights
json_file = open('model_3.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
optimizer = keras.optimizers.Adagrad(lr=.01,epsilon=0,decay=0)
model.compile(optimizer= optimizer ,loss='binary_crossentropy',metrics=['acc'])
model.load_weights('model_3.h5')

print("Loaded model weights from disk")

Loaded model weights from disk


## How we show performance?
##### The primary basis for measuring a model performance is measuring 'Log Loss' or basically a measure of accuracy that factors in confidence
##### Below we will show: Log Loss for random choice of winner/confidence, The Log Loss for the Top 5 models as part of the Kaggle NCAA competition for the 3 years we are testing
##### Link to Kaggle Leaderboard for 2019: https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard

In [26]:
def LogLoss(predictions, realizations):
    predictions_use = predictions.clip(0)
    realizations_use = realizations.clip(0)
    LogLoss = -np.mean( (realizations_use * np.log(predictions_use)) + 
                        (1 - realizations_use) * np.log(1 - predictions_use) )
    return LogLoss

In [27]:
#For reference this is the LogLoss of a model that predicts 50/50 shot of either team winning each time
#rather than favoring one team or another

bench_5050 = np.repeat(0.5, len(Y_test))
LogLoss(bench_5050, Y_test.AWon)

0.6931471805599452

In [28]:
#Log Loss for past Kaggle competition winners
#https://www.kaggle.com/c/mens-machine-learning-competition-2019/leaderboard
pastScores

,Year,Rank,LogLoss
0,2017,1,0.43857
1,2017,2,0.44981
2,2017,3,0.45373
3,2017,4,0.46100
4,2017,5,0.46107
5,2018,1,0.53194
6,2018,2,0.53693
7,2018,3,0.54013
8,2018,4,0.54967
9,2018,5,0.54987


## How does my model stack up?
##### Here is a readout of accuracy followed by Log Loss for the 3 years of Kaggle competition shown above
##### We then show a sample of outputs for the models outputted prediction/confidence

In [29]:
#LogLoss FN for accuracy weighted by confidence


#Accuracy on Test set
years=[2017,2018,2019]
for year in years:
    X_test_temp=X_test[X_test['Season']==year].reset_index(drop=True)
    Y_test_temp=Y_test[Y_test['Season']==year].reset_index(drop=True)
    Y_test_temp=Y_test_temp[['AWon']]
    
    scores = model.evaluate(X_test_temp.iloc[:,0:51], Y_test_temp, verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    #Test LogLoss on holdout test set
    y_pred_keras = model.predict(X_test_temp.iloc[:,0:51]).ravel()
    y_pred_round=np.round(y_pred_keras,0)
    temp = np.array(y_pred_keras)[np.newaxis]
    np.set_printoptions(suppress=True)
    y_pred_tran=temp.T
    print("LL_Test ",year,": ",LogLoss(y_pred_tran, Y_test_temp))


y_pred_keras = model.predict(X_test.iloc[:,0:51]).ravel()
y_pred_round=np.round(y_pred_keras,0)
testPerf=X_test.iloc[:,0:51]
testPerf['y_pred_confidence']=y_pred_keras

testPerf['y_pred_test']=y_pred_round
testPerf['y_act']=Y_test.AWon
testPerf.sample(10)

acc: 86.57%
LL_Test  2017 :  AWon    0.342599
dtype: float64
acc: 82.09%
LL_Test  2018 :  AWon    0.399666
dtype: float64
acc: 91.04%
LL_Test  2019 :  AWon    0.218012
dtype: float64


,A_KP_AdjEM,A_KP_SOS_Adj,ALuck,Asos,Ats_pct,Aie,Afour,Amov,APOM,ASAG,AMOR,Aelo,Awins_top25,Awins_top5,Awin_pct,AAdjustD,AAdjustO,B_KP_AdjEM,B_KP_SOS_Adj,BLuck,Bsos,Bts_pct,Bie,Bfour,Bmov,BPOM,BSAG,BMOR,Belo,Bwins_top25,Bwins_top5,Bwin_pct,BAdjustD,BAdjustO,KP_AdjEM_diff,KP_SOS_Adj_diff,Luck_diff,sos_diff,ts_pct_diff,ie_diff,four_diff,mov_diff,POM_diff,SAG_diff,MOR_diff,elo_diff,wins_top25_diff,wins_top5_diff,win_pct_diff,AdjustD_diff,AdjustO_diff,y_pred_confidence,y_pred_test,y_act
83,-0.391662,0.656176,1.215667,-0.862627,0.619964,-0.882955,-0.894251,-0.884379,0.048060,0.000924,0.077608,-0.890402,-0.191881,-0.556471,-0.847379,0.066522,-0.532625,-0.175277,0.639446,-1.050101,-0.697413,1.360860,-1.087825,-2.016709,-0.543738,-0.258058,-0.369400,0.026325,0.208095,-0.232025,1.221655,-1.094435,0.772286,0.402538,-0.156589,0.018836,1.596471,-0.129719,-0.552280,0.136423,0.893725,-0.245407,0.217342,0.263248,0.037634,-0.783518,0.030787,-1.187165,0.170461,-0.481230,-0.650872,0.970952,1.0,1
170,-0.193940,0.160500,-0.146061,-0.325564,0.607344,-0.317208,-0.704790,-0.245545,-0.243121,-0.272271,-0.263365,-1.034738,0.703894,-0.556471,-0.242957,0.342241,0.000341,-0.537639,-1.806350,0.847718,0.910301,1.048982,0.842571,-0.212679,1.139289,0.217313,0.326567,-0.109933,-0.286497,-1.099018,-0.539080,1.078182,0.772286,-0.120284,0.226030,1.351537,-0.690386,-0.860917,-0.333716,-0.811243,-0.370188,-0.962127,-0.334676,-0.435115,-0.113204,-0.527887,1.183551,-0.036743,-0.926083,-0.286990,0.083309,0.069368,0.0,0
190,-0.104683,-0.427878,-1.373848,-0.314172,0.941298,-0.682246,-1.355435,-0.452811,-0.165473,0.059466,0.077608,-0.800265,-0.639769,1.089378,-0.847379,0.480101,0.258227,-0.125865,0.370255,0.616276,-0.402559,0.654770,-0.799894,-0.758711,-0.436831,-0.278726,-0.451278,-0.324052,0.977163,-0.665522,-0.539080,-0.191635,0.894025,0.576812,0.011351,-0.552509,-1.410027,0.055544,0.191185,0.077058,-0.439000,-0.015877,0.072074,0.365093,0.284461,-1.276458,0.030787,1.113680,-0.461872,-0.273116,-0.217585,0.904224,1.0,0
182,0.904264,0.902121,0.389701,0.727519,1.280323,0.803167,-0.270623,0.841623,-0.670186,-0.662551,-0.680110,-1.553490,1.599669,1.089378,0.965887,0.302853,1.633622,-0.279986,0.877873,0.755141,-0.923243,0.975214,-1.046738,-0.118232,-0.860593,0.031298,0.121871,-0.187794,-1.458771,-0.232025,-0.539080,-0.879030,1.320111,0.733659,0.822536,0.026753,-0.242610,1.155165,0.198310,1.296150,-0.113602,1.188359,-0.523523,-0.580608,-0.360031,-0.047536,1.183551,1.113680,1.295146,-0.689344,0.636954,0.871390,1.0,1
168,1.420601,1.465905,-0.346972,0.856592,0.855059,0.756674,-0.129692,0.813978,-0.728422,-0.701579,-0.717996,1.240835,2.495444,-0.556471,0.717008,-0.937883,1.341350,1.437701,1.302810,0.014529,1.227511,0.181327,0.891277,0.047933,1.570723,-0.754098,-0.737853,-0.771757,1.351732,2.802447,4.743126,0.986755,-1.114665,1.186771,0.027987,0.128355,-0.258602,-0.241342,0.475305,-0.088339,-0.136077,-0.515695,-0.015087,0.001362,0.023922,-0.097143,-0.257404,-3.488011,-0.186752,0.101490,0.119416,0.757501,1.0,1
125,0.907653,0.586176,-0.235355,1.032597,2.426238,0.766838,0.189220,1.218155,-0.709010,-0.701579,-0.736938,-1.136976,1.151781,-0.556471,0.717008,0.283159,1.633622,0.492973,0.814420,-0.564074,0.018590,0.175192,0.306019,-0.074776,0.019606,-0.568083,-0.533157,-0.616034,-1.600941,1.068463,-0.539080,-0.271980,-0.810318,0.019136,0.304455,-0.150060,0.221158,0.720040,1.596239,0.327029,0.202316,0.843375,-0.131301,-0.144130,-0.099492,0.351446,0.030787,-0.036743,0.694884,0.753582,1.130419,0.908271,1.0,0
155,0.376629,1.552932,1.327284,-0.643146,-0.268881,-0.686768,-1.404178,-0.641712,-0.476065,-0.545467,-0.414909,0.697812,2.495444,2.735227,-0.162960,-0.524304,0.120688,-1.271774,-1.289118,0.755141,-0.737934,2.033803,-1.052722,-1.150834,-0.742609,0.940704,1.227229,0.668685,0.721883,-1.099018,-0.539080,-0.575505,1.746197,-0.381694,1.120393,1.967745,0.429054,0.054098,-1.674827,0.250213,-0.170029,0.063100,-1.017433,-1.278971,-0.771409,-0.026896,2.336315,2.264103,0.288456,-1.563424,0.348096,0.

# So that's cool, but how do we do at ACTUALLY picking a bracket?
#### We start with the 2017 bracket and show each rounds matchups as my model picked them, followed by the calculated ESPN score

In [30]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from runTournament import runTournament

df=pd.read_csv("tourney_game_master1.csv")
#2017 results
dfList, scoreDF=runTournament(df,2017,'model_3','model_3')
for df in dfList:
    print('\n',df)

scoreDF


                  ATeam              BTeam  ATeamID  BTeamID
0       mount-st-marys          villanova     1291     1437
1            wisconsin      virginia-tech     1458     1439
2       unc-wilmington           virginia     1423     1438
3              florida  east-tennessee-st     1196     1190
4                  usc                smu     1425     1374
5        new-mexico-st             baylor     1308     1124
6       south-carolina          marquette     1376     1266
7                 duke               troy     1181     1407
8      south-dakota-st            gonzaga     1355     1211
9           vanderbilt       northwestern     1435     1321
10           princeton         notre-dame     1343     1323
11            bucknell      west-virginia     1137     1452
12              xavier           maryland     1462     1268
13  florida-gulf-coast         florida-st     1195     1199
14                 vcu        saint-marys     1433     1388
15             arizona       north-dak

,round,score
0,32,29
1,16,24
2,8,24
3,4,16
4,2,0
5,1,0
6,Total,93


# 2017 might not have been the best year for us, how about 2018?
#### Same process as before

In [31]:
#2018 results
df=pd.read_csv("tourney_game_master1.csv")
dfList, scoreDF=runTournament(df,2018,'model_3','model_3')
for df in dfList:
    print('\n',df)

scoreDF


                     ATeam             BTeam  ATeamID  BTeamID
0               villanova           radford     1437     1347
1           virginia-tech           alabama     1439     1104
2               murray-st     west-virginia     1293     1452
3                marshall        wichita-st     1267     1455
4          st-bonaventure           florida     1382     1196
5              texas-tech  stephen-f-austin     1403     1372
6                  butler          arkansas     1139     1116
7        cal-st-fullerton            purdue     1168     1345
8                  kansas              penn     1242     1335
9       north-carolina-st        seton-hall     1301     1371
10                clemson     new-mexico-st     1155     1308
11  college-of-charleston            auburn     1158     1120
12                    tcu          syracuse     1395     1393
13            michigan-st          bucknell     1277     1137
14               oklahoma      rhode-island     1328     1348
15    

,round,score
0,32,29
1,16,22
2,8,16
3,4,16
4,2,16
5,1,32
6,Total,131


# We did ok considering a 16 seed upset a 1 seed in the 1st round 2018. Finally, this last year:

In [32]:
#2019 results
df=pd.read_csv("tourney_game_master1.csv")
dfList, scoreDF=runTournament(df,2019,'model_3','model_3')
for df in dfList:
    print('\n',df)

scoreDF



                 ATeam                BTeam  ATeamID  BTeamID
0     north-dakota-st                 duke     1295     1181
1                 vcu                  ucf     1433     1416
2      mississippi-st              liberty     1280     1251
3       virginia-tech          saint-louis     1439     1387
4            maryland              belmont     1268     1125
5                yale                  lsu     1463     1261
6           minnesota           louisville     1278     1257
7             bradley          michigan-st     1133     1277
8             gonzaga  fairleigh-dickinson     1211     1192
9            syracuse               baylor     1393     1124
10          marquette            murray-st     1266     1293
11            vermont           florida-st     1436     1199
12         arizona-st              buffalo     1113     1138
13         texas-tech    northern-kentucky     1403     1297
14            florida               nevada     1196     1305
15           michigan 

,round,score
0,32,28
1,16,30
2,8,32
3,4,24
4,2,16
5,1,32
6,Total,162


# 2019 went pretty well, but how well?
#### Below is my models score compared to the average ESPN score and the score given that you just pick higher seeds
#### Also shown is a percentile score compared to all other users on ESPN

In [33]:
pastMadness

,Year,avg_score,higherSeed_score,myScore,percentile_myScore
0,2017,65.7,82,93,83.00
1,2018,57.0,81,131,99.20
2,2019,74.0,124,162,99.99


# Now could this make me a millionaire?
#### Well, if I worked for Berkshire in 2019 according to my model and the stipulations, yes.
#### See this article: https://www.si.com/extra-mustard/2019/03/19/warren-buffett-ncaa-tournament-bracket-pool-million-dollars-year-sweet-16

# Finally, are bigger companies with lots of staff doing better?
#### Not quite, as far as I could find the only company publishing any kind of results is Adobe
#### Their model achieves performance that puts them in the 98th percentile, not quite 99.99...
#### See this page: https://www.adobe.com/analytics/hack-the-bracket.html?red=a